In [1]:
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network, load_data_to_gpu
from beamng_control import beamng_server, beamng_client
from pcdet_tools import beamng_point_cloud_process_tools, beamng_point_cloud_dataset, draw_scenes
from pcdet.utils import common_utils
import open3d as o3d
import numpy as np
import torch

import time

2023-06-12 20:43:45,501 |INFO     |beamngpy                      |Started BeamNGpy logging.


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
logger = common_utils.create_logger()

In [3]:
server_a = beamng_server()
client_b = beamng_client(logger=logger)

In [4]:
server_a.init_server()
server_a.launch_server()

2023-06-12 20:43:47,912 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_openpcdet_beamngpy\lib\site-packages\beamngpy\connection\connection.py:-1: ResourceWarning: unclosed <socket.socket fd=6004, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 5348)>

2023-06-12 20:43:47,912 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_openpcdet_beamngpy\lib\site-packages\beamngpy\connection\connection.py:-1: ResourceWarning: unclosed <socket.socket fd=6004, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 5348)>

2023-06-12 20:43:47,915 |INFO     |beamngpy.BeamNGpy             |Opening BeamNGpy instance.
2023-06-12 20:43:47,926 |INFO     |beamngpy.BeamNGpy             |Started BeamNG.
2023-06-12 20:43:57,928 |INFO     |beamngpy.BeamNGpy             |Connecting to BeamNG.tech at: (127.0.0.1, 64256)
2023-06-12 20:43:58,637 |INFO     |beamngpy.BeamNGpy             |Successfully connected to Beam

In [5]:
client_b.init_client()
client_b.launch_client()

2023-06-12 20:43:59,670 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2023-06-12 20:43:59,671 |INFO     |beamngpy.BeamNGpy             |BeamNGpy successfully connected to existing BeamNG instance.


In [6]:
server_a.debug_luanch_test()

2023-06-12 20:44:26,123 |INFO     |beamngpy.BeamNGpy             |Loaded map.
2023-06-12 20:44:28,058 |INFO     |beamngpy.Vehicle              |Vehicle vehicleA connected to simulation.
2023-06-12 20:44:28,058 |INFO     |beamngpy.BeamNGpy             |Attempting to connect to vehicle vehicleA
2023-06-12 20:44:29,073 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2023-06-12 20:44:29,074 |INFO     |beamngpy.BeamNGpy             |Successfully connected to vehicle vehicleA.
2023-06-12 20:44:29,078 |INFO     |beamngpy.Scenario             |Connected to scenario: tag
2023-06-12 20:44:29,100 |INFO     |beamngpy.BeamNGpy             |Starting scenario.


{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 69.118263244628906, 39.679920196533203, 16.415634155273438 ],
			"boundingbox_min" : [ -0.059999999999999998, -39.679874420166016, -6.9146575927734375 ],
			"field_of_view" : 60.0,
			"front" : [ -0.90307097537632919, 0.0017988087570628851, 0.42948757574567964 ],
			"lookat" : [ 34.529131622314452, 2.288818359375e-05, 4.75048828125 ],
			"up" : [ 0.42948904059539766, 0.0070563614983622357, 0.90304450154510629 ],
			"zoom" : 0.69999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

In [7]:
CFG_FILE = ".\\cfgs\\kitti_models\\pointpillar.yaml"
CKPT_FILE = "D:\\models_zoo\\pointpillar_7728.pth"

cfg = cfg_from_yaml_file(CFG_FILE, cfg)

client_b.debug_luanch_test(None)
pcs_dataset = beamng_point_cloud_dataset(dataset_cfg = cfg.DATA_CONFIG, logger=logger, lidar=client_b.lidar_t, class_names=cfg.CLASS_NAMES)
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=pcs_dataset)
model.load_params_from_file(filename=CKPT_FILE, logger=logger, to_cpu=True)
model.cuda()
model.eval()

2023-06-12 20:44:29,346 |INFO     |beamngpy.BeamNGpy             |Attempting to connect to vehicle vehicleA


tag
{'vehicleA': <beamngpy.vehicle.vehicle.Vehicle object at 0x000002446B233A00>}


2023-06-12 20:44:30,232 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2023-06-12 20:44:30,233 |INFO     |beamngpy.BeamNGpy             |Successfully connected to vehicle vehicleA.
2023-06-12 20:44:30,265 |INFO     |beamngpy.Lidar                |Opened lidar: "lidar"
2023-06-12 20:44:30,436 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_openpcdet_beamngpy\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

2023-06-12 20:44:30,436 |WARNING  |py.warnings                   |d:\anaconda3\envs\env_openpcdet_beamngpy\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered inte

PointPillar(
  (vfe): PillarVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): Linear(in_features=10, out_features=64, bias=False)
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (backbone_3d): None
  (map_to_bev_module): PointPillarScatter()
  (pfe): None
  (backbone_2d): BaseBEVBackbone(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d((1, 1, 1, 1))
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (8): BatchNorm2d(64, eps=0.001, moment

In [8]:
vis = o3d.visualization.Visualizer()
vis.create_window()

with torch.no_grad():
    for idx, data_dict in enumerate(pcs_dataset):
        data_dict = pcs_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)

        pred_dicts, _ = model.forward(data_dict)

        draw_scenes(vis,
                points=data_dict['points'][:, 1:], ref_boxes=pred_dicts[0]['pred_boxes'].detach(),
                ref_scores=pred_dicts[0]['pred_scores'].detach(), ref_labels=pred_dicts[0]['pred_labels'].detach(), confidence=0.5
            )
        

        vis.poll_events()
        vis.update_renderer()
        vis.clear_geometries()

[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
[Open3D WARNING] [ViewControl] SetView

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [ ]:
client_b.disconnect_client()


In [ ]:
server_a.close_server()